In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from binance.client import Client
from datetime import datetime
import time
import os
import requests

# --- Binance Setup ---
api_key = 'your_api_key'       # Replace with real key
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

symbol = "BTCUSDT"
DISCORD_WEBHOOK = 'https://discord.com/api/webhooks/1335150632759463936/uk95qYb2pMvfc_FjQURFIPD5AoQagifXgLLe_YbocWRiCaPxnXqLlBxA-qjIHgvhNt5v'

interval_map = {
    '1m': Client.KLINE_INTERVAL_1MINUTE,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '15m': Client.KLINE_INTERVAL_15MINUTE,
    '1h': Client.KLINE_INTERVAL_1HOUR
}

def generate_liq_heatmap(symbol, interval_label, interval_code):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval_code, limit=1000)
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'num_trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

        # --- Liquidation logic ---
        df['wick_size'] = (df['high'] - df['low']) / df['close']
        df['liq_score'] = df['volume'] * df['wick_size']
        df['liq_norm'] = (df['liq_score'] - df['liq_score'].min()) / (df['liq_score'].max() - df['liq_score'].min())

        df['color'] = df['liq_norm'].apply(lambda val: 'red' if val > 0.7 else 'yellow' if val > 0.4 else 'grey' if val > 0.1 else None)

        # --- EMA crossover logic ---
        df['ema_short'] = df['close'].ewm(span=50, adjust=False).mean()
        df['ema_long'] = df['close'].ewm(span=200, adjust=False).mean()
        df['golden_cross'] = (df['ema_short'] > df['ema_long']) & (df['ema_short'].shift(1) <= df['ema_long'].shift(1))
        df['death_cross'] = (df['ema_short'] < df['ema_long']) & (df['ema_short'].shift(1) >= df['ema_long'].shift(1))
        df['crossover_at_liq'] = False
        df['crossover_type'] = None

        high_liq = df[df['liq_norm'] > 0.7]
        for i in high_liq.index:
            idx = df.index.get_loc(i)
            window = df.iloc[max(0, idx-1): idx+2]
            if window['golden_cross'].any():
                df.at[i, 'crossover_at_liq'] = True
                df.at[i, 'crossover_type'] = 'Golden Cross'
            elif window['death_cross'].any():
                df.at[i, 'crossover_at_liq'] = True
                df.at[i, 'crossover_type'] = 'Death Cross'

        # --- Plotting ---
        chart_filename = f"{symbol}_{interval_label}_heatmap.png"
        plt.figure(figsize=(15, 8))
        plt.plot(df.index, df['close'], color='cyan', label='Price')
        plt.plot(df.index, df['ema_short'], color='blue', linestyle='--', alpha=0.6, label='EMA 50')
        plt.plot(df.index, df['ema_long'], color='orange', linestyle='--', alpha=0.6, label='EMA 200')

        for i in range(len(df)):
            row = df.iloc[i]
            color = row['color']
            if color:
                price_level = row['close']
                plt.axhline(y=price_level, color=color, alpha=0.3, linewidth=1)
                if row['crossover_at_liq']:
                    label = f"{row['crossover_type']} @ {price_level:.2f}"
                    plt.text(df.index[i], price_level, label, color='white',
                             fontsize=7, ha='left', va='bottom', bbox=dict(facecolor=color, alpha=0.3))

        plt.scatter(df.index[df['golden_cross']], df['close'][df['golden_cross']],
                    marker='^', color='green', label='Golden Cross', zorder=5)
        plt.scatter(df.index[df['death_cross']], df['close'][df['death_cross']],
                    marker='v', color='red', label='Death Cross', zorder=5)

        plt.title(f"{symbol} - {interval_label} Liquidation Heatmap", fontsize=14)
        plt.xlabel("Time")
        plt.ylabel("Price (USDT)")
        plt.legend()
        plt.grid(True, alpha=0.2)
        plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        plt.tight_layout()
        plt.savefig(chart_filename)
        plt.close()

        # --- Discord Alert ---
        with open(chart_filename, 'rb') as f:
            files = {'file': (chart_filename, f, 'image/png')}
            payload = {'content': f"**{symbol} - {interval_label.upper()} Liquidation Heatmap**\n"}
            response = requests.post(DISCORD_WEBHOOK, data=payload, files=files)
        os.remove(chart_filename)

        if response.status_code == 200:
            print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")
        else:
            print(f"[✘] Failed ({interval_label}): {response.status_code}")

    except Exception as e:
        print(f"[!] Error in {interval_label} for {symbol}: {str(e)}")


# --- Global 15-min loop ---
while True:
    for interval_label, interval_code in interval_map.items():
        generate_liq_heatmap(symbol, interval_label, interval_code)

    print("\n[⏳] Waiting 15 minutes for next cycle...\n")
    time.sleep(900)  # 15 min = 900 seconds


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_26380\3388263621.py:105: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 1m chart for BTCUSDT at 2025-06-30 15:01:25.146792


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_26380\3388263621.py:105: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 5m chart for BTCUSDT at 2025-06-30 15:01:27.564970


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_26380\3388263621.py:105: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 15m chart for BTCUSDT at 2025-06-30 15:01:29.535826


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_26380\3388263621.py:105: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 1h chart for BTCUSDT at 2025-06-30 15:01:31.809991

[⏳] Waiting 15 minutes for next cycle...



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from binance.client import Client
from datetime import datetime
import time
import os
import requests

# --- Binance Setup ---
api_key = 'your_api_key'       # Replace with real key
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

symbol = "BTCUSDT"
DISCORD_WEBHOOK = 'https://discord.com/api/webhooks/1335150632759463936/uk95qYb2pMvfc_FjQURFIPD5AoQagifXgLLe_YbocWRiCaPxnXqLlBxA-qjIHgvhNt5v'

interval_map = {
    '1m': Client.KLINE_INTERVAL_1MINUTE,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '15m': Client.KLINE_INTERVAL_15MINUTE,
    '1h': Client.KLINE_INTERVAL_1HOUR
}
def generate_liq_heatmap(symbol, interval_label, interval_code):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval_code, limit=1000)
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'num_trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].astype(float)

        # --- Liquidation intensity (same idea as before)
        df['wick_size'] = (df['high'] - df['low']) / df['close']
        df['liq_score'] = df['volume'] * df['wick_size']
        if df['liq_score'].max() == df['liq_score'].min():
            df['liq_norm'] = 0.0
        else:
            df['liq_norm'] = (df['liq_score'] - df['liq_score'].min()) / (df['liq_score'].max() - df['liq_score'].min())

        # --- Color buckets unchanged (you can tweak thresholds)
        def bucket(val):
            if val > 0.7: return 'red'
            if val > 0.4: return 'yellow'
            if val > 0.1: return 'grey'
            return None
        df['color'] = df['liq_norm'].apply(bucket)

        # --- EMAs & cross detection
        df['ema_short'] = df['close'].ewm(span=50, adjust=False).mean()
        df['ema_long']  = df['close'].ewm(span=200, adjust=False).mean()
        df['golden_cross'] = (df['ema_short'] > df['ema_long']) & (df['ema_short'].shift(1) <= df['ema_long'].shift(1))
        df['death_cross']  = (df['ema_short'] < df['ema_long']) & (df['ema_short'].shift(1) >= df['ema_long'].shift(1))
        df['crossover_at_liq'] = False
        df['crossover_type'] = None

        high_liq = df[df['liq_norm'] > 0.7]
        for i in high_liq.index:
            idx = df.index.get_loc(i)
            window = df.iloc[max(0, idx-1): idx+2]
            if window['golden_cross'].any():
                df.at[i, 'crossover_at_liq'] = True
                df.at[i, 'crossover_type'] = 'Golden Cross'
            elif window['death_cross'].any():
                df.at[i, 'crossover_at_liq'] = True
                df.at[i, 'crossover_type'] = 'Death Cross'

        # --- Normalize volume for styling & scoring
        vol_min, vol_max = df['volume'].min(), df['volume'].max()
        if vol_max == vol_min:
            df['vol_norm'] = 0.0
        else:
            df['vol_norm'] = (df['volume'] - vol_min) / (vol_max - vol_min)

        # --- ATR for distance-to-level normalization (proximity)
        tr1 = (df['high'] - df['low']).abs()
        tr2 = (df['high'] - df['close'].shift()).abs()
        tr3 = (df['low']  - df['close'].shift()).abs()
        df['tr'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df['atr14'] = df['tr'].rolling(14).mean()

        last_close = float(df['close'].iloc[-1])
        last_atr = float(df['atr14'].iloc[-1]) if not pd.isna(df['atr14'].iloc[-1]) else (df['high'] - df['low']).rolling(14).mean().iloc[-1]
        if not pd.notna(last_atr) or last_atr == 0:
            last_atr = (df['high'] - df['low']).mean()

        # --- Scored liquidity levels (top N)
        # Use only rows that actually produced a colored line
        liq_rows = df[df['color'].notna()].copy()
        if not liq_rows.empty:
            liq_rows['price_level'] = liq_rows['close']
            # proximity: 1 when at price, falling as distance grows; floor at 0
            liq_rows['prox_norm'] = (1 - (liq_rows['price_level'].sub(last_close).abs() / (3 * last_atr))).clip(lower=0, upper=1)

            # Blend score (tune weights as you like)
            w_liq, w_vol, w_prox = 0.5, 0.3, 0.2
            liq_rows['hit_score'] = (w_liq * liq_rows['liq_norm']) + (w_vol * liq_rows['vol_norm']) + (w_prox * liq_rows['prox_norm'])

            top_levels = liq_rows.sort_values('hit_score', ascending=False).head(6)[
                ['price_level','volume','liq_norm','vol_norm','prox_norm','hit_score','crossover_type']
            ]
        else:
            top_levels = pd.DataFrame(columns=['price_level','volume','liq_norm','vol_norm','prox_norm','hit_score','crossover_type'])

        # --- Plotting (price + EMAs + VOLUME on secondary axis)
        chart_filename = f"{symbol}_{interval_label}_heatmap.png"
        fig, ax_price = plt.subplots(figsize=(15, 8))

        # price and EMAs
        ax_price.plot(df.index, df['close'], label='Price', linewidth=1.2)
        ax_price.plot(df.index, df['ema_short'], linestyle='--', alpha=0.7, label='EMA 50')
        ax_price.plot(df.index, df['ema_long'],  linestyle='--', alpha=0.7, label='EMA 200')

        # volume bars on secondary axis (no custom colors; defaults are fine)
        ax_vol = ax_price.twinx()
        ax_vol.bar(df.index, df['volume'], alpha=0.25, width=0.8)  # width auto adjusted by matplotlib
        ax_vol.set_ylabel("Volume")
        ax_vol.grid(False)  # keep primary grid only

        # draw horizontal levels; thickness/alpha reflect volume
        for i in range(len(df)):
            row = df.iloc[i]
            if row['color'] is None:
                continue
            level = float(row['close'])
            volw  = float(row['vol_norm']) if pd.notna(row['vol_norm']) else 0.0
            lw    = 0.5 + 2.5 * volw    # 0.5 to 3.0
            alpha = 0.15 + 0.55 * volw  # 0.15 to 0.70
            ax_price.axhline(y=level, alpha=alpha, linewidth=lw, color=row['color'])

            if bool(row.get('crossover_at_liq', False)):
                label = f"{row.get('crossover_type','Cross')} @ {level:.2f}"
                ax_price.text(df.index[i], level, label, fontsize=7, ha='left', va='bottom')

        # mark crosses
        ax_price.scatter(df.index[df['golden_cross']], df['close'][df['golden_cross']], marker='^', s=30, label='Golden Cross')
        ax_price.scatter(df.index[df['death_cross']],  df['close'][df['death_cross']],  marker='v', s=30, label='Death Cross')

        ax_price.set_title(f"{symbol} - {interval_label} Liquidation Heatmap (Volume-weighted)")
        ax_price.set_xlabel("Time")
        ax_price.set_ylabel("Price (USDT)")
        ax_price.legend(loc='upper left')
        ax_price.grid(True, alpha=0.2)
        ax_price.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        fig.tight_layout()
        fig.savefig(chart_filename)
        plt.close(fig)

        # --- Build Discord text table of top levels
        def fmt_row(r):
            return f"{r['price_level']:.2f} | vol {r['volume']:.0f} | score {r['hit_score']:.2f}" + \
                   (f" | {r['crossover_type']}" if isinstance(r['crossover_type'], str) else "")

        if not top_levels.empty:
            lines = "\n".join(fmt_row(r) for _, r in top_levels.iterrows())
        else:
            lines = "No strong liquidity levels found."

        content = (
            f"**{symbol} - {interval_label.upper()} Liquidation Heatmap**\n"
            f"Price: {last_close:.2f}\n"
            f"Top levels (price | vol | score):\n```\n{lines}\n```"
        )

        # --- Discord post
        with open(chart_filename, 'rb') as f:
            files = {'file': (chart_filename, f, 'image/png')}
            payload = {'content': content}
            response = requests.post(DISCORD_WEBHOOK, data=payload, files=files)
        os.remove(chart_filename)

        if response.status_code == 200:
            print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")
        else:
            print(f"[✘] Failed ({interval_label}): {response.status_code}")

    except Exception as e:
        print(f"[!] Error in {interval_label} for {symbol}: {str(e)}")

# --- Global 15-min loop ---
while True:
    for interval_label, interval_code in interval_map.items():
        generate_liq_heatmap(symbol, interval_label, interval_code)

    print("\n[⏳] Waiting 15 minutes for next cycle...\n")
    time.sleep(900)  # 15 min = 900 seconds



C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_824\597228568.py:177: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 1m chart for BTCUSDT at 2025-09-24 10:32:24.710860


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_824\597228568.py:177: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 5m chart for BTCUSDT at 2025-09-24 10:32:26.999740


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_824\597228568.py:177: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 15m chart for BTCUSDT at 2025-09-24 10:32:29.962912


C:\Users\dhirajsv\AppData\Local\Temp\ipykernel_824\597228568.py:177: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[✔] Sent {interval_label} chart for {symbol} at {datetime.utcnow()}")


[✔] Sent 1h chart for BTCUSDT at 2025-09-24 10:32:33.229721

[⏳] Waiting 15 minutes for next cycle...

